### Improving an estimator for the BetMGM's Over/Under
 My current prediction method takes into account offensive rating, defensive rating, and pace of play
 However, it does not take into account lineup changes.
 Here is an example of where this can be an issue:
 The Phoenix Suns - Orlando Magic current over/under is set at 209.5 by BetMGM (see Yahoo Sports)
 This is much lower than my guess


In [12]:
import pandas as pd
import nba_api.stats.endpoints as e
import warnings
warnings.filterwarnings('ignore')

In [13]:
A = e.LeagueDashTeamStats(season="2024-25",measure_type_detailed_defense="Advanced",date_from_nullable='2024-10-22',date_to_nullable='2024-11-18').get_data_frames()[0]
A.loc[:,['TEAM_ID','TEAM_NAME','GP','W','L','MIN','OFF_RATING','DEF_RATING','PACE','POSS']].set_index("TEAM_ID").iloc[[21,23]]

,TEAM_NAME,GP,W,L,MIN,OFF_RATING,DEF_RATING,PACE,POSS
TEAM_ID,,,,,,,,,
1610612753,Orlando Magic,14,8,6,672.0,108.3,103.5,98.75,1379
1610612756,Phoenix Suns,14,9,5,682.0,112.3,113.4,98.46,1400


In [14]:
o1,o2,d1,d2,p1,p2 = 108.3,112.3,103.5,113.4,98.75,98.46
p = (p1+p2)/2
o = (o1+o2)/2
d = (d1+d2)/2
estimator = p*(o+d)/100
print(estimator)

215.69843749999995



 There are many factors that my guess is not taking into account, one of those being lineup changes
 For the Magic, Banchero has been out since Oct 31 and Wendell Carter Jr since Nov 3rd
 For the Suns, Durant has been out since Nov 8th and Bradley Beal since Nov 12
 Let's try and take this into account by changing how I get the offensive ratings, defensive ratings, and paces for each team.
 I can adjust the date_to and date_from parameters to obtain more recent data only


In [15]:
A = e.LeagueDashTeamStats(season="2024-25",measure_type_detailed_defense="Advanced",date_from_nullable='2024-11-03',date_to_nullable='2024-11-18').get_data_frames()[0]
A.loc[:,['TEAM_ID','TEAM_NAME','GP','W','L','MIN','OFF_RATING','DEF_RATING','PACE','POSS']].set_index("TEAM_ID").iloc[[21,23]]

,TEAM_NAME,GP,W,L,MIN,OFF_RATING,DEF_RATING,PACE,POSS
TEAM_ID,,,,,,,,,
1610612753,Orlando Magic,8,5,3,384.0,106.2,99.4,97.25,776
1610612756,Phoenix Suns,8,4,4,389.0,112.8,117.7,97.17,788


In [16]:
o1,o2,d1,d2,p1,p2 = 106.2,112.8,99.4,117.7,97.25,97.17
o = (o1+o2)/2
d = (d1+d2)/2
p = (p1+p2)/2
estimator = p*(o+d)/100
print(estimator)

211.966405



 As you can see, our new guess for over/under is a little better.
 But there are some other factors coming into play here
 One, both teams are traveling west. The Suns played in Minnesota yesterday, and the Magic are coming off a homestand in Orlando
 Moving all the way to Phoenix Arizona and changing timezones should affect them, at least a little bit.
 So the discrepancy that's left between 211.96 and the current line (209.5) can be due to multiple factors
 It could be due to sharp bettors trying to get in early on the under bet, or it could be due to the sportsbooks calculating in timezone travel and days rest, among other factors
 As a NBA betting nerd, I am excited to find out what actually happens.
 If anything, I expect each team to have about 97 possessions during regulation time